## Sección 7.8 Generales de probabilidad

8. Se lanzan simultáneamente 4 monedas. Determine la probabilidad de obtener dos caras y dos sellos $\mathbb{P}(A) = 3/8$. Realice el cálculo de esta probabilidad usando un experimento virtual con $N = 10^{5}$ eventos, y etiquetando los resultados con $+1$ y $-1$ para cara y sello respectivamente.

In [125]:
import numpy as np
import random

N = 10**5

doscaras_dossellos = 0

for i in range(N):
    cara_sello = np.zeros(4)
    for i in range(4):
        cara_sello[i] = random.choice([-1,1])
    if sum(cara_sello)==0:
        doscaras_dossellos +=1 
round(doscaras_dossellos/N,4)

0.3762

## Sección 7.12 Distribuciones continuas de probabilidad

1. Dada la función de probabilidad conjunta:

\begin{equation*}
f(x,y) = \left \{ \begin{aligned} \frac{2}{3}(x+2y) &\ 0 \leq x \leq 1,0 \leq y \leq 1 \\ 0 \ &(\text{en otro caso})
\end{aligned} 
\right .
\end{equation*}

encuentre analíticamente y a través del paquete SymPy los siguientes valores:

a) Verifique que sea una función de densidad conjunta válida.

b) Hallar las distribuciones marginales g(x) y h(y).

c) $\mathbb{E}(x) = \frac{10}{18}$

d) $\mathbb{E}(y) = \frac{11}{18}$

e) Calcular la covarianza usando: $\sigma_{xy} = \mathbb{E}(xy) - \mathbb{E}(x)\mathbb{E}(y) = -0.00617$

f) Calcular la covarianza usando: $\sigma_{xy} = \mathbb{E}((x-\hat{\mu}_x)(y-\hat{\mu}_y)) = -0.00617$

g) Son las variables x e y independientes?

In [82]:
from scipy.integrate import quad
import sympy as sym

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

fxy = sym.Piecewise(((2/3)*(x+2*y),(x>=0)&(x <=1)&(y>=0)&(y<=1)),(0,True))
fxy

Piecewise((0.666666666666667*x + 1.33333333333333*y, (x >= 0) & (y >= 0) & (x <= 1) & (y <= 1)), (0, True))

In [83]:
#a)
dfxy = sym.integrate(fxy,(x,0,1),(y,0,1))
dfxy

1.00000000000000

In [84]:
#b)
gx = sym.integrate(fxy,(y,-sym.oo,sym.oo))
gx

Piecewise((0.666666666666667*x + 0.666666666666667, (x >= 0) & (x <= 1)), (0, True))

In [85]:
hy = sym.integrate(fxy,(x,-sym.oo,sym.oo))
hy

Piecewise((1.33333333333333*y + 0.333333333333333, (y >= 0) & (y <= 1)), (0, True))

In [112]:
#c)
Ex = sym.integrate(x*gx,(x,-sym.oo,sym.oo))
Ex

0.555555555555556

In [113]:
#d)
Ey = sym.integrate(y*hy,(y,-sym.oo,sym.oo))
Ey

0.611111111111111

In [119]:
#e)
Exy = sym.integrate(x*y*gx*hy,(x,-sym.oo,sym.oo),(y,-sym.oo,sym.oo))
Exy-Ex*Ey

0

In [115]:
#f)
sym.integrate((x-Ex)*(y-Ey),(x,0,1),(y,0,1))

0.00617283950617281

In [121]:
#g)
print("Sí son independientes.")

Sí son independientes.


## Sección 7.18 Hidden Markov models

1. Un casino tiene dos tipos de monedas una justa $(J)$ y una sesgada $(B)$, las probabilidades de obtener cara y sello son: $\mathbb{P}_J=[0.5,0.5]$ y $\mathbb{P}_B=[0.9,0.1]$ respectivamente. El tipo de moneda se escoge siguiendo esta ley de transición: 

\begin{equation*}
\mathbb{T} = \begin{pmatrix}
 & J & B\\
J & 0.8 & 0.2\\
B & 0.2 & 0.8
\end{pmatrix}
\end{equation*}

Por otro lado, la matriz de emisión está dada por:

\begin{equation*}
\mathbb{E} = \begin{pmatrix}
 & J & B\\
S & 0.5 & 0.9\\
C & 0.5 & 0.1
\end{pmatrix}
\end{equation*}

Se realiza un experimento de 8 lanzamientos y se obtiene la siguiente secuencia:

\begin{equation*}
\Omega_O = [S,C,C,C,S,C,S,C]
\end{equation*}

a) Use la siguiente distribución de probabilidad a-priori $\pi = [0.2,0.8]$ para la moneda justa y sesgada.

b) Encuentre la secuencia oculta más probable del tipo de moneda que se eligió en cada lanzamiento y su respectiva probabilidad $\mathbb{P}_i$.

c) Calcule las probabilidades de cada estado observable $(o)$ como la suma de las
probabilidades de todos los estados ocultos, $\mathbb{P}_o=\sum_i\mathbb{P}_i$.

d) Verifique que la suma de todos los estados observables es 1.

\begin{equation*}
\sum\mathbb{P}_o = 1
\end{equation*}

e) ¿Depende el resultado de cada probabilidad a-priori?

In [63]:
import numpy as np
#a)

pi = np.array([0.2,0.8])
T = np.array([[0.8,0.2],\
              [0.2,0.8],\
             ])
E = np.array([[0.5,0.9],\
              [0.5,0.1],\
             ])

#b)
Dict_coin = { 0: 'J', 1: 'B'}
Dict_ht = { 0: 'C', 1: 'S'}

def GetStates(Tansition,Emition,T,N=int(8)):
    
    coin_type = ""
    heads_tails = ""
    States = np.array(Tansition)
    
    for i in range(N):
        T = np.dot(T,Tansition)
        States = np.vstack((States,T))
        index = np.where(np.amax(T)==T)[0]
        
        coin = Dict_coin[index[0]]
        coin_type = coin_type + coin

        if coin == 'J':
            index = np.where(np.amax(Emition[:,0])==Emition[:,0])[0]
        else:
            index = np.where(np.amax(Emition[:,1])==Emition[:,1])[0]
        ht_iteration = Dict_ht[index[0]]
        heads_tails = heads_tails + ht_iteration 

        
    return coin_type,heads_tails,States

coin_type,heads_tails, States = GetStates(T,E,pi)
print(coin_type,heads_tails)

#c)

#d)

#e)

BBBBBBBB CCCCCCCC


## Sección 9.4 Mínimos Cuadrados

1. Realice una búsqueda iterativa entre $-5\leq x \leq 5$ y $-5 \leq y \leq 5$ con un paso de $h = 0.01$ para encontrar la menor distancia del problema. Grafique la distancia y compare con el resultado obtenido con mínimos cuadrados.

7. Calcule la proyección ortogonal del vector $\vec{b} = (-3,-3,8,9)$ sobre el sub-espacio W generado por los vectores:

\begin{align*}
\vec{u_1}&=(3,1,0,1)\\
\vec{u_2}&=(1,2,1,1)\\
\vec{u_3}&=(-1,0,2,-1)
\end{align*}

a) Usando mínimos cuadrados matriciales. La proyección ortogonal es $p_W(b)=Ax$, donde las columnas de A son los vectores base y x es la solución de mínimos cuadrados.

b) Con el proceso de Grand-Schmidt obtener una base ortonormal $(v_1,v_2,v_3)$ y luego calcular la proyección sobre dicha base: $p_W(b)=c_1v_1+c_2v_2+c_3v_3$, donde $c_i=<b.v_i>$ para $i=1,2,3$. Respuesta: $p_W(b)=(-2,3,4,0)$

Recuerde que el procedimiento de Grand-Schmidt es:

\begin{equation*}
u_k = v_k-\sum^{k-1}_{j=1}\frac{<v_k,u_j>}{<u_j,u_j>}u_j
\end{equation*}

In [64]:
import numpy as np
#a)
u1=[3,1,0,1]
u2=[1,2,1,1]
u3=[-1,0,2,-1]
A = np.array([u1,u2,u3]).T
b = np.array([-3,-3,8,9])

At = A.T
M = np.dot(At,A)
bt = np.dot(At,b)
xsol = np.linalg.solve(M,bt)
pWb = np.dot(A,xsol)

print(pWb)

#b)
M = np.zeros((At.shape),dtype=np.float64)
M[0] = At[0]
for i in range(1,At.shape[0]):
    M[i]=At[i]
    for j in range(i):
        M[i] = M[i] -(np.dot(At[i],M[j]))/(np.dot(M[j],M[j]))*M[j]
v1 = M[0]
v2 = M[1]
v3 = M[2]

pWb = np.dot(b,v1)*v1 + np.dot(b,v2)*v2 + np.dot(b,v3)*v3
print(pWb)

[-2.0000000e+00  3.0000000e+00  4.0000000e+00 -4.4408921e-16]
[-13.61765675   9.42223981  14.25980207  -1.56926957]


## Sección 9.7 Estimación de Parámetros

1. Sea $\mathscr{A}(x_1,x_2,...,x_n)$, donde $\mathscr{A} \sim \mathscr{N}(\mu,\sigma)$ con parámetros $\mu$ y $\sigma$. Muestre que los estimadores verosímiles son:

\begin{align*}
\hat{\mu} &= \frac{1}{n} \sum^n_{i=1}x_i \\
\hat{\sigma}^2 &= \frac{1}{n} \sum^n_{i=1}(x_i-\bar{x})^2
\end{align*}

#### Parte 1. Estimador verosímil de $\mu$

Partimos de la función de verosimilitud, que da como resultado lo siguiente.

\begin{equation*}
L(\mu) = \prod^n_{i=1} \frac{1}{\sqrt{2\pi\sigma^2}}e^{\frac{1}{2\sigma^2}(x_i-\mu)^2}
\end{equation*}

Simplificando esta función, obtenemos lo siguiente.

\begin{align*}
L(\mu) &= \left(\frac{1}{\sqrt{2\pi\sigma^2}}\right)^n e^{\frac{-1}{2\sigma^2}\sum^n_{i=1}(x_i-\mu)^2}\\
L(\mu) &= (2\pi\sigma^2)^{\frac{-n}{2}} e^{\frac{-1}{2\sigma^2}\sum^n_{i=1}(x_i-\mu)^2}\\
\end{align*}

Ahora tomando el logaritmo natural de la función de verosimilitud se obtiene lo siguiente.

\begin{align*}
\ln(L(\mu)) &= \ln(2\pi\sigma^2)^{\frac{-n}{2}} + \ln\left(e^{\frac{-1}{2\sigma^2}\sum^n_{i=1}(x_i-\mu)^2}\right)\\
\ln(L(\mu)) &= \frac{-n}{2}\ln(2\pi\sigma^2) -\frac{1}{2\sigma^2}\sum^n_{i=1}(x_i-\mu)^2\ln(e)
\end{align*}

Al maximizar la función obtenida, el parámetro máximo verosímil para el parámetro $\mu$ da como resultado lo siguiente.

\begin{align*}
\frac{\partial\ln(L(\mu))}{\partial\mu} &= 0\\
\frac{2}{2\sigma^2}\sum^n_{i=1}(x_i-\mu) &= 0\\
\sum^n_{i=1}x_i-\sum^n_{i=0}\mu&=0\\
\sum^n_{i=1}x_i-n\mu&=0
\end{align*}

De lo cual el estimador verosímil de $\mu$ da como resultado la siguiente función.

\begin{equation*}
\hat{\mu} = \frac{1}{n}\sum^n_{i=1}x_i
\end{equation*}

#### Parte 2. Estimador verosímil de $\sigma^2$

Partimos de la función obtenida para el logaritmo natural de la función de verosimilitud. Al maximizar la función obtenida, el parámetro máximo verosímil para el parámetro $\sigma^2$ da como resultado lo siguiente.

\begin{align*}
\frac{\partial\ln(L(\sigma))}{\partial\sigma} &= 0\\
\frac{-n}{\sigma} +\frac{1}{\sigma^3}\sum^n_{i=1}(x_i-\mu)^2 &= 0\\
\frac{1}{\sigma^3}\sum^n_{i=1}(x_i-\mu)^2 &=\frac{n}{\sigma}\\
\frac{1}{n}\sum^n_{i=1}(x_i-\mu)^2 &=\sigma^2
\end{align*}

Teniendo en cuenta que el estimador verosímil de $\mu$ es equivalente a la media muestral, el estimador verosímil de $\sigma^2$ da como resultado la siguiente función.

\begin{equation*}
\hat{\sigma}^2 = \frac{1}{n}\sum^n_{i=1}(x_i-\bar{x})^2
\end{equation*}